## **Import libraries**

In [0]:
import os
import subprocess
import os
import zipfile
import cv2
import numpy as np
import os
import zipfile
import torch
from moviepy.video.io.VideoFileClip import VideoFileClip
from google.colab.patches import cv2_imshow
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils
%matplotlib inline
import os
from PIL import Image
device = "cuda" if torch.cuda.is_available() else "cpu"

# **Convert label to emotion category**

In [0]:
def convertLabel(label):
    if(label==0):
        return "Anger"
    if(label==1):
        return "Disgust"
    if(label==2):
        return "Fear"
    if(label==3):
        return "Happy"
    if(label==4):
        return "Neutral"     
    if(label==5):
        return "Sad"
    if(label==6):
        return "Surprise"

# **Paths**

In [0]:
# input video path
videoPath = "test.avi"
detector_path = "/cascade/deploy.prototxt.txt"
detector_path2 = "/cascade/res10_300x300_ssd_iter_140000.caffemodel"

## **Extract 5 face and background images from video**

In [0]:
detector = cv2.dnn.readNetFromCaffe(detector_path, detector_path2)

if __name__ == '__main__' :        
    cap = cv2.VideoCapture(videoPath);
    clip = VideoFileClip(videoPath)
            
    cap.set(cv2.CAP_PROP_FRAME_COUNT, 400)
    length=50
    count=0
    name_count=1
    print("Pre-processing Video......")
    while(cap.isOpened()):
        count+=1
        if length ==count:
            break
        if name_count==6:
            break
        ret, img = cap.read()
        if img is None:
            continue
        inputBlob = cv2.dnn.blobFromImage(cv2.resize(img, (224, 224)), 1, (300, 300), (104, 177, 123))
        detector.setInput(inputBlob)
        detections = detector.forward()
        
        for i in range(0, detections.shape[2]):
            # Probability of prediction
            prediction_score = detections[0, 0, i, 2]
            if prediction_score < 0.6:
                continue
            # Finding height and width of frame
            (h, w) = img.shape[:2]
            # compute the (x, y)-coordinates of the bounding box for the
            # object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")
            
            if y2 > h or x2 > w:
                continue
            if y2 <= y1 or x2 <= x1:
                continue
            if y2 ==0 or x2 ==0 or y1 ==0 or x1 ==0:
                continue
            if y2 -y1<=0 or x2 - x1<=0:
                continue
            if y1 < 0 or y2<0 or x1<0 or x2<0:
                continue

            img2 = img[y1:y2, x1:x2]
            face_name = 'temp/Face_' +str(name_count)+ '.jpg'               #
            cv2.imwrite(face_name, img2)
            blank_image = np.zeros(shape=[y2-y1, x2-x1, 3], dtype=np.uint8)
            img[y1:y2, x1:x2] = blank_image
            background_name = 'temp/Background_' +str(name_count)+ '.jpg'         #
            name_count+=1
            cv2.imwrite(background_name, img)
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    if(name_count!=6):
        print("Face can't be detected in the video!")
    else:
        print("Video Pre-processing Completed!")

Pre-processing Video......
Video Pre-processing Completed!


In [0]:
data_transforms = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [0]:
X1=[]
for i in range(1,6):
          img_name = 'temp/Face_' +str(i)+ '.jpg'
          image = Image.open(str(img_name))
          image = data_transforms(image)
          X1.append(image.squeeze_(0))
X2 = []
for i in range(1,6):
          img_name = 'temp/Background_' +str(i)+ '.jpg' 
          image = Image.open(str(img_name))
          image = data_transforms(image)
          X2.append(image.squeeze_(0))

X1 = torch.stack(X1, dim=0).unsqueeze_(0)

X2 = torch.stack(X2, dim=0).unsqueeze_(0)

x1=X1.squeeze(1)
x1=x1.transpose(1,2)
x1=x1.type(torch.cuda.FloatTensor)

x2=X2.squeeze(1)
x2=x2.transpose(1,2)
x2=x2.type(torch.cuda.FloatTensor)


## **Convert video .mp4 to .wav**

In [0]:
command="ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = videoPath, output = 'temp/output')
subprocess.call(command, shell=True)
command ="ffmpeg -i 'output.mp4' -ab 320k -ac 2 -ar 44100 -vn '{}.wav'".format('temp/output')    
subprocess.call(command, shell=True)

0

# **Extract audio feature**

In [0]:
songname = 'temp/output.wav'
audio_feature=[]
y, sr = librosa.load(songname, mono=True, duration=7)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
rmse = librosa.feature.rmse(y=y)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)
audio_feature.append(np.mean(chroma_stft))
audio_feature.append(np.mean(rmse))
audio_feature.append(np.mean(spec_cent))
audio_feature.append(np.mean(spec_bw))
audio_feature.append(np.mean(rolloff))
audio_feature.append(np.mean(zcr))
for e in mfcc:
    audio_feature.append(np.mean(e))
audio = np.array(audio_feature)
audio = audio.astype('float').reshape(-1, 26)
audio = torch.from_numpy(audio)
audio = audio.type(torch.cuda.FloatTensor)
audio = audio.to(device)

# **Build model and load pretrained parameters**

In [0]:
PATH = "/model/Audio-3D-CNN.pt"
from model.Audio_3D_CNN import Model, Face, Context, Audio
model = Model(Face(),Context(),Audio())
model.load_state_dict(torch.load(PATH))
model.to(device)
model.eval()

Model(
  (modelA): Face(
    (conv1): Conv3d(3, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool2): MaxPool3d(kernel_size=(1, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv3d(64, 128, kernel_size=(1, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool3): MaxPool3d(kernel_size=(1, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv4): Conv3d(128, 256, kernel_size=(1, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool4): MaxPoo

# **Predict the emotion**

In [0]:
output=model(x1,x2,audio)
print(convertLabel(np.argmax(output.cpu().detach().numpy())))

tensor([[-1.0906, -2.9325, -3.1102, -3.1126, -1.6156, -2.6584, -1.3751]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
Anger
